# Compute climate index TG ('Mean daily mean temperature'). The computation in the back is taken from the python module xclim

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import index_calculator as xcalc
import xarray as xr

Select one CORDEX `tas` file and open it with xarray.

In [ ]:
ifile='/pool/data/CORDEX/data/cordex/output/EUR-11/GERICS/MPI-M-MPI-ESM-LR/historical/r3i1p1/GERICS-REMO2015/v1/day/tas/v20190925/tas_EUR-11_MPI-M-MPI-ESM-LR_historical_r3i1p1_GERICS-REMO2015_v1_day_20010101-20051231.nc'

In [ ]:
def open_xrdataset(files, use_cftime=True, parallel=True, data_vars='minimal', chunks={'time':1}, 
                   coords='minimal', compat='override', drop=None, **kwargs):
    """optimized function for opening large cf datasets.
    based on https://github.com/pydata/xarray/issues/1385#issuecomment-561920115
    decode_timedelta=False is added to leave variables and coordinates with time units in 
    {“days”, “hours”, “minutes”, “seconds”, “milliseconds”, “microseconds”} encoded as numbers.   
    """
    def drop_all_coords(ds):
        return ds.reset_coords(drop=True)

    ds = xr.open_mfdataset(files, parallel=parallel, decode_times=False, combine='by_coords', 
                           preprocess=drop_all_coords, decode_cf=False, chunks=chunks,
                           data_vars=data_vars, coords=coords, compat=compat, **kwargs)

    return xr.decode_cf(ds, use_cftime=use_cftime, decode_timedelta=False)

In [ ]:
ds = open_xrdataset(ifile)
ds

Now do a preprocessing of the dataset and create an index_calculator.PreProcessing object.
Preprocessor tasks are:
* select a user-given time range (parameter: time_range)
* check time axis for missing, redundant and duplicated time steps
* crop time axis to required left and right border
* write available and requested time range to object

In [ ]:
preproc = xcalc.preprocessing(ds)
preproc.preproc

Now, use the above object to compute the climate index `TG`.

In [ ]:
idx_ds = xcalc.processing('TG', preproc_obj=preproc)
idx_ds.proc

Finally, do some postprocessing. The postprocessor writes some additional attributes to `ds`' global attributes.

In [ ]:
postproc = xcalc.postprocessing('CORDEX', proc_obj=idx_ds, institution='Helmholtz-Zentrum hereon GmbH, Climate Service Center Germany', 
             institution_id='GERICS')
postproc.postproc

Optionaly, we can store the dataset as a netCDF file on disk. The output file naming depends on the user-given project. 

In [ ]:
xcalc.outputwriter(postproc_obj=postproc).write_to_netcdf()

For simplyfication, we can do preprocessing, processing, postprocessing and output file storing by calloing just one function.

In [ ]:
idx_ds = xcalc.index_calculator(ds=ds, 
                                index='TG', 
                                project='CORDEX', 
                                institution='GERICS', 
                                institution_id='Helmholtz-Zentrum hereon GmbH, Climate Service Center Germany').compute(write=True)

In [ ]:
idx_ds